<a href="https://colab.research.google.com/github/ESSI-Lab/dab-pynb/blob/main/dab-py_demo_whos_hydrosos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Welcome to the Jupyter notebook showing programmatically discovery and access of WHOS categorical data - for HydroSOS integration**

This notebook is used to programmatically access WHOS DAB functionalities, in this case through the OGC OM-JSON based API, documented and available for tests also here: https://whos.geodab.eu/gs-service/om-api

The WHOS portal can also be used to help debug your python notebook: https://whos.geodab.eu/gs-service/whos/search.html

## **Step 1: Install dab-py**

The Python client library `dab-py` is available on PyPI and GitHub: https://github.com/ESSI-Lab/dab-py

In [2]:
!pip install --upgrade dab-py

## **Step 2: Main Code Tutorial**

### **2.1: Import Library**
Import the main `dabpy` library to access WHOS API `om_api` functionalities.

In [3]:
from dabpy import WHOSClient, Constraints
from IPython.display import display

### **2.2: Initialize WHOS Client (Token & View)**
Replace with your WHOS API token and optional view. This initializes the client to interact with the WHOS API.

As a prerequisite to obtain programmatically access to WHOS, a token is required. It can be obtained after registration at https://whos.geodab.eu/gs-service/whos/registration.html.

In [4]:
from google.colab import userdata
# Replace with your WHOS API token and optional view
token = userdata.get('token-whos')  # replace with your actual token
view = "whos"
client = WHOSClient(token=token, view=view)

### **2.3: Define Feature Constraints**
Feature retrieval in the WHOS API can be restricted using optional constraints.

#### 2.3.1: Spatial Constraints *(bounding box)*  
Set the geographic bounding box for spatial queries. The example uses a bbox around Argentina.

In [1]:
# Define bounding box coordinates (south, west, north, east)
south = -54
west = -72
north = -22
east = -53

In this example, only spatial constraints (bounding box) are applied, while the other filters remain optional.

In [5]:
# Create feature constraints
constraints = Constraints(bbox = (south, west, north, east))

#### 2.3.2: Optional: Other Constraints
Additional optional constraints can be applied to further filter the results:
- **Observed property** : filter features based on the type of measurement or variable, such as water level, precipitation, discharge, etc.
- **Ontology**          : specify which ontology to use to expand the observed property search term or URI with synonyms and related narrower concepts. Two ontologies are available: `whos` and `his-central`.
- **Country**           : using Country code (ISO3).
- **Provider**          : data provider identifier.
- The other constraints are listed from the documented API https://whos.geodab.eu/gs-service/om-api/

A very important constraint in this case is the timeInterpolation, representing the type of data interpolation applied (e.g. average, min, max, ...). In case of categorical data used for hydrological status report, just use "categorical":

In [17]:
constraints = Constraints(
    bbox = (south, west, north, east),
    timeInterpolation = "categorical"
 )

# You can also specify additional constraints as needed, for example:
'''
constraints = Constraints(
    bbox = (south, west, north, east),
    observedProperty = "example_property",
    ontology = "example_ontology",
    country = "example_country",
    provider = "example_provider",
    spatialRelation = "example_spatial_relation",
    feature = "example_feature_id",                       # can also use features[index_number].id directly
    localFeatureIdentifier = "example_local_id",
    observationIdentifier = "example_obs_id",
    beginPosition = "2025-01-01T00:00:00Z",
    endPosition = "2025-02-01T00:00:00Z",
    predefinedLayer = "example_layer",
    timeInterpolation = "example_interpolation",
    intendedObservationSpacing = "PT1H",                  # ISO 8601 duration, e.g., 1 hour
    aggregationDuration = "P1D",                          # ISO 8601 duration, e.g., 1 day
    limit = "10",                                         # by default, it will only gives the first 10 data entries, if you want more, specify here.
    format = "format_type"                                # specify the format type if needed, e.g., "NetCDF", "GeoTIFF", etc.
)
'''

'\nconstraints = Constraints(\n    bbox = (south, west, north, east),\n    observedProperty = "example_property",\n    ontology = "example_ontology",\n    country = "example_country",\n    provider = "example_provider",\n    spatialRelation = "example_spatial_relation",\n    feature = "example_feature_id",                       # can also use features[index_number].id directly\n    localFeatureIdentifier = "example_local_id",\n    observationIdentifier = "example_obs_id",\n    beginPosition = "2025-01-01T00:00:00Z",\n    endPosition = "2025-02-01T00:00:00Z",\n    predefinedLayer = "example_layer",\n    timeInterpolation = "example_interpolation",\n    intendedObservationSpacing = "PT1H",                  # ISO 8601 duration, e.g., 1 hour\n    aggregationDuration = "P1D",                          # ISO 8601 duration, e.g., 1 day\n    limit = "10",                                         # by default, it will only gives the first 10 data entries, if you want more, specify here.\n    form

### **2.4: Get `Features`**

#### 2.4.1: Retrieve features as Python objects
Fetch features from the WHOS API URL as Python `Feature` objects using the previously defined constraints.

In [33]:
# 01.1.1: Get Features as Python objects
features = client.get_features(constraints)

Retrieving page 1: https://whos.geodab.eu/gs-service/services/essi/token/***/view/whos/om-api/features?west=-72&south=-54&east=-53&north=-22&timeInterpolation=categorical
Returned first 10 features (not completed, more data available).
Use .next() to move to the next page.


The retrieved **features** are stored in a `FeaturesCollection`.  
Then convert the current page to a Pandas DataFrame for easier inspection and analysis.then converted into a Pandas DataFrame for easier inspection, filtering, and analysis.

In [34]:
# 01.1.2: (optional: Convert Features to DataFrame if needed)
features_df = features.to_df()
display(features_df)

,ID,Name,Coordinates,Source,Identifier,Contact Name,Contact Email
0,015727FD7702C31AD1C3993BBFE6CA3ED3424011,Barranqueras,"-58.9333333333333, -27.4833333333333","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:20,,
1,0227F7805F9574A3687FB6ED398B867DACD739AE,La Emilia,"-60.3346777777778, -33.3989194444444","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_bdhi:133,,
2,0551D95BD0DC15B933AF0FC8F2A242EC9C87F267,Garruchos,"-55.65, -28.1833333333333","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:67,,
3,0630853B40B02230DFB24D97E796CCBCA816BF47,Paso de los Libres,"-57.0833333333333, -29.7166666666667","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:72,,
4,06A1AEFB5D1C92C1A1620AA652BB6CFDB4C96600,Los Laureles,"-59.2171666666667, -29.7571111111111","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_bdhi:121,,
5,0704E13E016926196EB6DF379C40FAC49B19E0DE,Posadas,"-55.8833333333333, -27.3666666666667","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:14,,
6,18001080BE61A68263854C69A2355559ED158C42,San Pedro,"-59.6497183332918, -33.6745267329027","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:38,,
7,28427569966654D717D0FD7778427726F510AD7D,Cnel. Bogado,"-60.576775, -33.3600611111111","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_bdhi:140,,
8,2EA2492AA5D4AB74C8D9A507669D3EB3908DACE6,Paraná - Itatí,"-58.2443055555556, -27.26625","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_bdhi:2082,,
9,45EC676BAF66D33BC43B0DFB46BB789199ED6FE7,Santo Tomé,"-56.0333333333333, -28.55","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:68,,


#### 2.4.2: (If needed / more data exists) Fetch the next page
If the dataset is **not completed** (contains more than 10 datas) - like the above data, fetch the next page using `.next()`

In [35]:
# 01.2.1: Fetch next page (if available).
nextFeatures = features.next()

# 01.2.2: (optional) Convert current page features to DataFrame.
nextFeatures_df = nextFeatures.to_df()
display(nextFeatures_df)

Retrieving page 2: https://whos.geodab.eu/gs-service/services/essi/token/***/view/whos/om-api/features?west=-72&south=-54&east=-53&north=-22&timeInterpolation=categorical&resumptionToken=45EC676BAF66D33BC43B0DFB46BB789199ED6FE7
Returned next 10 features (not completed, more data available).
Use .next() to move to the next page.


,ID,Name,Coordinates,Source,Identifier,Contact Name,Contact Email
0,4A56CA42BC9787F1C1386473D746070B28D7A303,Pueblo Andino,"-60.8659444444444, -32.6733333333333","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_bdhi:118,,
1,4C8611CA7B43CD3A160BF0217CD851A98182A78F,Salto Grande Arriba,"-57.936945, -31.27547","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:77,,
2,4E4EF6CFB5CA20C1C1A5A258042DA1152979A140,Paso Ledesma,"-57.6745111111111, -29.8458","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_bdhi:130,,
3,623DF8F9163692B11C2E06B13369814CBC0770C6,Santa Lucía,"-59.0982222222222, -28.9911111111111","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_bdhi:107,,
4,67DA63C1ADA572777445105F814C0E1E39868C2D,Santa Fe,"-60.7002319185745, -31.6514772196376","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:30,,
5,6F0F1559D58FA751EB1CC6D48DE07C58E07CEAD2,Corrientes,"-58.8388696, -27.46364349","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:19,,
6,7CBF10EC8C00DE2A7FA888C2710E7C9FE2DE8B44,Yacyretá efluente,"-56.727244, -27.482557","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:88,,
7,850A8EE1D63326D5A8DD85ECC865D632C4FA89FA,Puerto Iguazú,"-54.5666666666667, -25.5833333333333","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:9,,
8,97971638E85607C46323CA8E2759D4DE6210401B,El Soberbio,"-54.2, -27.3","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:61,,
9,AD38CD86B07F4BD9C1EE7FDC1444FE79C6934116,Puerto Pilcomayo,"-57.65, -25.3666666666667","Argentina, Instituto Nacional del Agua (INA)",argentina-ina:alturas_prefe:55,,


### **2.5: Get `Observations`**


Here we select observations from a given station, the first in the list of stations, maintaining all other constraints.

If the dataset is not completed (contains more than 10 datas), fetch the next page using `.next()` *(see Step 2.4.2)*.

#### 2.5.2: Retrieve observations as Python objects (using the already defined constraints)

In [53]:
# 02.2.1: (or) retrieve observations from a different constraints - by defining new_constraints.
stationId = features[0].id
print(stationId)
constraints.feature = stationId
constraints.limit = 1

observations = client.get_observations(constraints)

015727FD7702C31AD1C3993BBFE6CA3ED3424011
Retrieving page 1: https://whos.geodab.eu/gs-service/services/essi/token/***/view/whos/om-api/observations?west=-72&south=-54&east=-53&north=-22&feature=015727FD7702C31AD1C3993BBFE6CA3ED3424011&timeInterpolation=categorical&limit=1
Returned first 1 observations (not completed, more data available).
Use .next() to move to the next page.


The retrieved **observations** are then converted into a Pandas DataFrame for easier inspection, filtering, and analysis.

In [54]:
# 02.2.2: (optional: Convert Observations to DataFrame if needed)
observations_df = observations.to_df()
display(observations_df)

,ID,Source,Observed Property,Phenomenon Time Begin,Phenomenon Time End
0,8DBAD70DD8645786737F750875D49B71AEE89A8C,"Argentina, Instituto Nacional del Agua (INA)","Discharge, stream",1990-04-01T03:00:00Z,2025-10-01T03:00:00Z


### **2.6: Get the Data Points**
#### 2.6.1: Retrieve Observation With Data Points

Retrieve the full time-series data for a selected observation within a specified time range. It shows the actual code that retrieves the data, to separate the action (API call) from what to do next with the data.

In [55]:
print(observations[0].id)
# 03.1: Get first observation with data points
obs_with_data = client.get_observation_with_data(observations[0].id, begin="2025-01-01", end="2026-01-01")

8DBAD70DD8645786737F750875D49B71AEE89A8C
Retrieving https://whos.geodab.eu/gs-service/services/essi/token/***/view/whos/om-api/observations?includeData=true&observationIdentifier=8DBAD70DD8645786737F750875D49B71AEE89A8C&beginPosition=2025-01-01&endPosition=2026-01-01
No observation data available for the requested time range.


#### 2.6.2: (optional) Convert to DataFrame for analysis

The returned data can be plotted or further analyzed as needed.

In [15]:
# 03.2: (optional: Convert Observation Points to DataFrame if needed)
obs_points_df = client.points_to_df(obs_with_data)
display(obs_points_df)

,Time,Value


#### 2.6.3: Example of Graphical Time-Series
This step demonstrates visualization, which is a different action from simply retrieving or tabulating data.  

In [16]:
# 03.3: (optional: Example of Graphical Time-Series)
client.plot_observation(obs_with_data, "")

No data points available for this observation.
